### N-Grams

In [ ]:
import re
from nltk.util import ngrams

In [ ]:
s = "Camera quality is not bad as expected"
s2 = "Unlike the previous reviews, I feel the screen touch lock is not bad"

In [ ]:
s = s.lower()
s = re.sub(r'[^a-zA-Z0-9\s]', '', s)
tokens = [token for token in s.split(" ") if token != ""]
output = list(ngrams(tokens, 2))

In [ ]:
s2 = "Unlike the previous reviews, I feel the screen touch lock is not bad"
s2 = s2.lower()
s2= re.sub(r'[^a-zA-Z0-9\s]', '', s2)
tokens = [token for token in s2.split(" ") if token != ""]
output2 = list(ngrams(tokens, 2))

In [ ]:
output2

###  Data Preparation for Segmentation

In [3]:
#imports
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [5]:
# read data into a DataFrame
data = pd.read_csv("reviews.csv")

#make a copy of columns I need from raw data
df1 = data.iloc[:, [3,4,5,6,8]]

In [6]:
df1 = df1.rename(columns={"stars":"Score","text":"Text"})
df1.head()

Score                                               Text    type  \
0      5  My wife took me here on my birthday for breakf...  review   
1      5  I have no idea why some people give bad review...  review   
2      4  love the gyro plate. Rice is so good and I als...  review   
3      5  Rosie, Dakota, and I LOVE Chaparral Dog Park!!...  review   
4      5  General Manager Scott Petello is a good egg!!!...  review   

                  user_id  useful  
0  rLtl8ZkDX5vH5nAx9C3q5Q       5  
1  0a2KyEL0d3Yb1V6aivbIuQ       0  
2  0hT2KtfLiobPvh6cDC8JQg       1  
3  uZetl9T0NcROGOyFfughhg       2  
4  vYmM4KTsC8ZfQBg-j5MWkw       0

In [ ]:
df1['Bad'] = np.where(df1.Score<=4,1,0)

In [ ]:
# convert text to lowercase
df1.loc[:, 'Text'] = df1['Text'].str.lower()

In [ ]:
#remove punctuation
import unicodedata
import sys

tbl = dict.fromkeys(i for i in range(sys.maxunicode)
                      if unicodedata.category(chr(i)).startswith('P'))
def remove_punctuation(text):
    return text.translate(tbl)

df1['Text']=df1['Text'].apply( lambda x: remove_punctuation(x))
df1["Text"].head(4)

In [ ]:
#tokenize text with Tfidf
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(min_df = 0.1, max_df=0.9,
                             ngram_range=(1, 4), 
                             stop_words='english')
vectorizer.fit(df1['Text'])

In [ ]:
X_train = vectorizer.transform(df1['Text'])
vocab = vectorizer.get_feature_names()

### Segmentation

In [7]:
#Apply TfidfVectorizer to review text
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn import metrics

In [9]:
# we create a kmeans model
model = KMeans(n_clusters=6, 
               init='k-means++', 
               max_iter=100, n_init=1,random_state=5)

In [10]:
# get the tf-idf table 
vectorizer = TfidfVectorizer(min_df = 0.05, max_df=0.95,
                             ngram_range=(1, 2), 
                             stop_words='english')
vectorizer.fit(df1['Text'])

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=0.95, max_features=None, min_df=0.05,
        ngram_range=(1, 2), norm='l2', preprocessor=None, smooth_idf=True,
        stop_words='english', strip_accents=None, sublinear_tf=False,
        token_pattern='(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=True,
        vocabulary=None)

In [13]:
# Create a training data
X_train = vectorizer.transform(df1['Text'])

In [14]:
# apply the model on this prepared data
res = model.fit(X_train)
res

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=100,
    n_clusters=6, n_init=1, n_jobs=1, precompute_distances='auto',
    random_state=5, tol=0.0001, verbose=0)

In [15]:
# Getting the vocalbulary as well as cluster centers
# Firstly, set of words from the tf-idf itself
vocab = vectorizer.get_feature_names()
vocab = np.array(vocab) 

In [21]:
# Then, the cluster centers from the model fit that we stored on res
cluster_centers = np.array(res.cluster_centers_)

In [32]:
#x=np.array((0.1,10,0.05,1.5))
#print(x)
#print(x.argsort())
res.cluster_centers_[0].argsort()

array([  4, 142,  83,  48, 135,  79, 110, 148,  73,   5,  88,  81,  94,
        67, 129,  20,   1,  56,   7, 149, 133,  55, 112, 143,  45, 124,
        64,  38,  41,  17,  89, 125, 114,  29,  23,  62,  58, 132, 138,
       100, 131, 130,   9, 117,  65, 127,  84, 153,  61,  18, 104,  60,
       106, 121, 140,   0,  97, 120,  32, 145,  72,  40,  27,  75,  21,
       102, 134, 147,  80,  16, 128,  22,  93, 155, 150, 139,  28,  24,
        90,  14,  47,  43,  35,  57,  33, 116,  70,  66, 105,  74,  30,
       126, 151,  96, 154,  15,  78,  92,   3, 101,  34, 122,  13, 108,
        77,  91, 111, 137,  37,  50, 115,  11,  49,  59, 146, 109,  85,
        95,  52,  39,  82,  10,  25,  71, 113,  44,  54,  19,  87,  31,
         2,  36, 152, 118,   8, 103, 144,  26,  69, 141,  12,  86, 107,
        63, 136,   6, 123,  46,  68,  76,  51,  42, 119,  99,  98,  53])

In [17]:
# sorting the cluster centers
sorted_vals = [res.cluster_centers_[i].argsort() for i in range(0,np.shape(res.cluster_centers_)[0])]

In [18]:
# get top 10 words from that cluster
words=set()
for i in range(len(res.cluster_centers_)):
    words = words.union(set(vocab[sorted_vals[i][-10:]]))
    print(words)

{'place', 'staff', 'good', 'food', 'great', 'pizza', 'friendly', 'service', 'love', 'like'}
{'place', 'staff', 'delicious', 'good', 'food', 'great', 'just', 'pizza', 'friendly', 'best', 'service', 'restaurant', 'love', 'like'}
{'place', 'great', 'pizza', 'friendly', 'salad', 'lunch', 'staff', 'delicious', 'good', 'food', 'sandwich', 'chicken', 'just', 'ordered', 'best', 'service', 'restaurant', 'love', 'like'}
{'place', 'great', 'nice', 'pizza', 'friendly', 'salad', 'little', 'really', 'lunch', 'pretty', 'staff', 'delicious', 'good', 'food', 'sandwich', 'chicken', 'just', 'ordered', 'best', 'service', 'restaurant', 'love', 'like'}
{'place', 'time', 'great', 'nice', 'pizza', 'friendly', 'salad', 'little', 'really', 've', 'store', 'lunch', 'pretty', 'staff', 'delicious', 'good', 'food', 'sandwich', 'chicken', 'just', 'ordered', 'best', 'amazing', 'service', 'restaurant', 'love', 'like'}
{'place', 'time', 'don', 'great', 'nice', 'pizza', 'friendly', 'salad', 'little', 'really', 've', 'sto

In [19]:
words=list(words)

In [ ]:
#add top words to train set
#train_set=X_train[:,[np.argwhere(vocab==i)[0][0] for i in words]]

In [34]:
#train_set[,]
#df1.head()

Score                                               Text    type  \
0      5  My wife took me here on my birthday for breakf...  review   
1      5  I have no idea why some people give bad review...  review   
2      4  love the gyro plate. Rice is so good and I als...  review   
3      5  Rosie, Dakota, and I LOVE Chaparral Dog Park!!...  review   
4      5  General Manager Scott Petello is a good egg!!!...  review   

                  user_id  useful  cluster  
0  rLtl8ZkDX5vH5nAx9C3q5Q       5        4  
1  0a2KyEL0d3Yb1V6aivbIuQ       0        0  
2  0hT2KtfLiobPvh6cDC8JQg       1        3  
3  uZetl9T0NcROGOyFfughhg       2        4  
4  vYmM4KTsC8ZfQBg-j5MWkw       0        5

In [35]:
# how many observations are in each cluster

df1['cluster'] = model.labels_
df1.groupby('cluster').count()

Score  Text  type  user_id  useful
cluster                                    
0         1165  1165  1165     1165    1165
1         1427  1427  1427     1427    1427
2          788   788   788      788     788
3         1387  1387  1387     1387    1387
4         2791  2791  2791     2791    2791
5         2442  2442  2442     2442    2442

In [36]:
# what does each cluster look like
df1.groupby('cluster').mean()

Score    useful
cluster                    
0        4.234335  1.071245
1        3.536790  1.131745
2        3.802030  1.422589
3        3.776496  1.229272
4        4.027589  1.233608
5        3.407043  2.031532